### Windowing

#### Programming Model

The key idea in Structured Streaming is to treat a live data stream as a table that is being continuously appended. This leads to a new stream processing model that is very similar to a batch processing model. You will express your streaming computation as standard batch-like query as on a static table, and Spark runs it as an incremental query on the unbounded input table. Let’s understand this model in more detail.
Basic Concepts

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

<img src='structured-streaming-stream-as-a-table.png'>

A query on the input will generate the “Result Table”. Every trigger interval (say, every 1 second), new rows get appended to the Input Table, which eventually updates the Result Table. Whenever the result table gets updated, we would want to write the changed result rows to an external sink.

<img src='structured-streaming-model.png'>

The “Output” is defined as what gets written out to the external storage. The output can be defined in a different mode:

1. Complete Mode - The entire updated Result Table will be written to the external storage. It is up to the storage connector to decide how to handle writing of the entire table.

2. Append Mode - Only the new rows appended in the Result Table since the last trigger will be written to the external storage. This is applicable only on the queries where existing rows in the Result Table are not expected to change.

3. Update Mode - Only the rows that were updated in the Result Table since the last trigger will be written to the external storage (available since Spark 2.1.1). Note that this is different from the Complete Mode in that this mode only outputs the rows that have changed since the last trigger. If the query doesn’t contain aggregations, it will be equivalent to Append mode.

Note that each mode is applicable on certain types of queries. This is discussed in detail later.

To illustrate the use of this model, let’s understand the model in context of the Quick Example above. The first lines DataFrame is the input table, and the final wordCounts DataFrame is the result table. Note that the query on streaming lines DataFrame to generate wordCounts is exactly the same as it would be a static DataFrame. However, when this query is started, Spark will continuously check for new data from the socket connection. If there is new data, Spark will run an “incremental” query that combines the previous running counts with the new data to compute updated counts, as shown below.

<img src='structured-streaming-example-model.png'>
    

Note that Structured Streaming does not materialize the entire table. It reads the latest available data from the streaming data source, processes it incrementally to update the result, and then discards the source data. It only keeps around the minimal intermediate state data as required to update the result (e.g. intermediate counts in the earlier example).

This model is significantly different from many other stream processing engines. Many streaming systems require the user to maintain running aggregations themselves, thus having to reason about fault-tolerance, and data consistency (at-least-once, or at-most-once, or exactly-once). In this model, Spark is responsible for updating the Result Table when there is new data, thus relieving the users from reasoning about it. As an example, let’s see how this model handles event-time based processing and late arriving data.

### Handling Event-time and Late Data

Event-time is the time embedded in the data itself. For many applications, you may want to operate on this event-time. For example, if you want to get the number of events generated by IoT devices every minute, then you probably want to use the time when the data was generated (that is, event-time in the data), rather than the time Spark receives them. This event-time is very naturally expressed in this model – each event from the devices is a row in the table, and event-time is a column value in the row. This allows window-based aggregations (e.g. number of events every minute) to be just a special type of grouping and aggregation on the event-time column – each time window is a group and each row can belong to multiple windows/groups. Therefore, such event-time-window-based aggregation queries can be defined consistently on both a static dataset (e.g. from collected device events logs) as well as on a data stream, making the life of the user much easier.

Furthermore, this model naturally handles data that has arrived later than expected based on its event-time. Since Spark is updating the Result Table, it has full control over updating old aggregates when there is late data, as well as cleaning up old aggregates to limit the size of intermediate state data. Since Spark 2.1, we have support for watermarking which allows the user to specify the threshold of late data, and allows the engine to accordingly clean up old state. These are explained later in more detail in the Window Operations section.

### Fault Tolerance Semantics

Delivering end-to-end exactly-once semantics was one of key goals behind the design of Structured Streaming. To achieve that, we have designed the Structured Streaming sources, the sinks and the execution engine to reliably track the exact progress of the processing so that it can handle any kind of failure by restarting and/or reprocessing. Every streaming source is assumed to have offsets (similar to Kafka offsets, or Kinesis sequence numbers) to track the read position in the stream. The engine uses checkpointing and write-ahead logs to record the offset range of the data being processed in each trigger. The streaming sinks are designed to be idempotent for handling reprocessing. Together, using replayable sources and idempotent sinks, Structured Streaming can ensure end-to-end exactly-once semantics under any failure.

### API using Datasets and DataFrames

Since Spark 2.0, DataFrames and Datasets can represent static, bounded data, as well as streaming, unbounded data. Similar to static Datasets/DataFrames, you can use the common entry point SparkSession (Scala/Java/Python/R docs) to create streaming DataFrames/Datasets from streaming sources, and apply the same operations on them as static DataFrames/Datasets. If you are not familiar with Datasets/DataFrames, you are strongly advised to familiarize yourself with them using the DataFrame/Dataset Programming Guide.

### Creating streaming DataFrames and streaming Datasets

Streaming DataFrames can be created through the DataStreamReader interface (Scala/Java/Python docs) returned by SparkSession.readStream(). In R, with the read.stream() method. Similar to the read interface for creating static DataFrame, you can specify the details of the source – data format, schema, options, etc.
Input Sources

There are a few built-in sources.

1. File source - Reads files written in a directory as a stream of data. Files will be processed in the order of file modification time. If latestFirst is set, order will be reversed. Supported file formats are text, CSV, JSON, ORC, Parquet. See the docs of the DataStreamReader interface for a more up-to-date list, and supported options for each file format. Note that the files must be atomically placed in the given directory, which in most file systems, can be achieved by file move operations.

2. Kafka source - Reads data from Kafka. It’s compatible with Kafka broker versions 0.10.0 or higher. See the Kafka Integration Guide for more details.

3. Socket source (for testing) - Reads UTF8 text data from a socket connection. The listening server socket is at the driver. Note that this should be used only for testing as this does not provide end-to-end fault-tolerance guarantees.

4. Rate source (for testing) - Generates data at the specified number of rows per second, each output row contains a timestamp and value. Where timestamp is a Timestamp type containing the time of message dispatch, and value is of Long type containing the message count, starting from 0 as the first row. This source is intended for testing and benchmarking.
    
5. Rate Per Micro-Batch source (for testing) - Generates data at the specified number of rows per micro-batch, each output row contains a timestamp and value. Where timestamp is a Timestamp type containing the time of message dispatch, and value is of Long type containing the message count, starting from 0 as the first row. Unlike rate data source, this data source provides a consistent set of input rows per micro-batch regardless of query execution (configuration of trigger, query being lagging, etc.), say, batch 0 will produce 0~999 and batch 1 will produce 1000~1999, and so on. Same applies to the generated time. This source is intended for testing and benchmarking.

Some sources are not fault-tolerant because they do not guarantee that data can be replayed using checkpointed offsets after a failure. See the earlier section on fault-tolerance semantics. Here are the details of all the sources in Spark.


### Window Operations on Event Time

Aggregations over a sliding event-time window are straightforward with Structured Streaming and are very similar to grouped aggregations. In a grouped aggregation, aggregate values (e.g. counts) are maintained for each unique value in the user-specified grouping column. In case of window-based aggregations, aggregate values are maintained for each window the event-time of a row falls into. Let’s understand this with an illustration.

Imagine our quick example is modified and the stream now contains lines along with the time when the line was generated. Instead of running word counts, we want to count words within 10 minute windows, updating every 5 minutes. That is, word counts in words received between 10 minute windows 12:00 - 12:10, 12:05 - 12:15, 12:10 - 12:20, etc. Note that 12:00 - 12:10 means data that arrived after 12:00 but before 12:10. Now, consider a word that was received at 12:07. This word should increment the counts corresponding to two windows 12:00 - 12:10 and 12:05 - 12:15. So the counts will be indexed by both, the grouping key (i.e. the word) and the window (can be calculated from the event-time).

The result tables would look something like the following.

<img scr='structured-streaming-window.png'>


### Handling Late Data and Watermarking

Now consider what happens if one of the events arrives late to the application. For example, say, a word generated at 12:04 (i.e. event time) could be received by the application at 12:11. The application should use the time 12:04 instead of 12:11 to update the older counts for the window 12:00 - 12:10. This occurs naturally in our window-based grouping – Structured Streaming can maintain the intermediate state for partial aggregates for a long period of time such that late data can update aggregates of old windows correctly, as illustrated below.

<img src='structured-streaming-late-data.png'>

However, to run this query for days, it’s necessary for the system to bound the amount of intermediate in-memory state it accumulates. This means the system needs to know when an old aggregate can be dropped from the in-memory state because the application is not going to receive late data for that aggregate any more. To enable this, in Spark 2.1, we have introduced watermarking, which lets the engine automatically track the current event time in the data and attempt to clean up old state accordingly. You can define the watermark of a query by specifying the event time column and the threshold on how late the data is expected to be in terms of event time. For a specific window ending at time T, the engine will maintain state and allow late data to update the state until (max event time seen by the engine - late threshold > T). In other words, late data within the threshold will be aggregated, but data later than the threshold will start getting dropped (see later in the section for the exact guarantees). Let’s understand this with an example. We can easily define watermarking on the previous example using withWatermark() as shown below.


In this example, we are defining the watermark of the query on the value of the column “timestamp”, and also defining “10 minutes” as the threshold of how late is the data allowed to be. If this query is run in Update output mode (discussed later in Output Modes section), the engine will keep updating counts of a window in the Result Table until the window is older than the watermark, which lags behind the current event time in column “timestamp” by 10 minutes. Here is an illustration.

<img src='structured-streaming-watermark-update-mode.png'>


As shown in the illustration, the maximum event time tracked by the engine is the blue dashed line, and the watermark set as (max event time - '10 mins') at the beginning of every trigger is the red line. For example, when the engine observes the data (12:14, dog), it sets the watermark for the next trigger as 12:04. This watermark lets the engine maintain intermediate state for additional 10 minutes to allow late data to be counted. For example, the data (12:09, cat) is out of order and late, and it falls in windows 12:00 - 12:10 and 12:05 - 12:15. Since, it is still ahead of the watermark 12:04 in the trigger, the engine still maintains the intermediate counts as state and correctly updates the counts of the related windows. However, when the watermark is updated to 12:11, the intermediate state for window (12:00 - 12:10) is cleared, and all subsequent data (e.g. (12:04, donkey)) is considered “too late” and therefore ignored. Note that after every trigger, the updated counts (i.e. purple rows) are written to sink as the trigger output, as dictated by the Update mode.

Some sinks (e.g. files) may not supported fine-grained updates that Update Mode requires. To work with them, we have also support Append Mode, where only the final counts are written to sink. This is illustrated below.

Note that using withWatermark on a non-streaming Dataset is no-op. As the watermark should not affect any batch query in any way, we will ignore it directly.


<img src='structured-streaming-watermark-append-mode.png'>


Similar to the Update Mode earlier, the engine maintains intermediate counts for each window. However, the partial counts are not updated to the Result Table and not written to sink. The engine waits for “10 mins” for late date to be counted, then drops intermediate state of a window < watermark, and appends the final counts to the Result Table/sink. For example, the final counts of window 12:00 - 12:10 is appended to the Result Table only after the watermark is updated to 12:11.

### Types of time windows

Spark supports three types of time windows: tumbling (fixed), sliding and session.

<img src='structured-streaming-time-window-types.jpg'>


Tumbling windows are a series of fixed-sized, non-overlapping and contiguous time intervals. An input can only be bound to a single window.

Sliding windows are similar to the tumbling windows from the point of being “fixed-sized”, but windows can overlap if the duration of slide is smaller than the duration of window, and in this case an input can be bound to the multiple windows.

Tumbling and sliding window use window function, which has been described on above examples.

Session windows have different characteristic compared to the previous two types. Session window has a dynamic size of the window length, depending on the inputs. A session window starts with an input, and expands itself if following input has been received within gap duration. For static gap duration, a session window closes when there’s no input received within gap duration after receiving the latest input.

Session window uses session_window function. The usage of the function is similar to the window function.


Instead of static value, we can also provide an expression to specify gap duration dynamically based on the input row. Note that the rows with negative or zero gap duration will be filtered out from the aggregation.

With dynamic gap duration, the closing of a session window does not depend on the latest input anymore. A session window’s range is the union of all events’ ranges which are determined by event start time and evaluated gap duration during the query execution.


Note that there are some restrictions when you use session window in streaming query, like below:

1. “Update mode” as output mode is not supported.
2. There should be at least one column in addition to session_window in grouping key.

For batch query, global window (only having session_window in grouping key) is supported.

By default, Spark does not perform partial aggregation for session window aggregation, since it requires additional sort in local partitions before grouping. It works better for the case there are only few number of input rows in same group key for each local partition, but for the case there are numerous input rows having same group key in local partition, doing partial aggregation can still increase the performance significantly despite additional sort.

You can enable spark.sql.streaming.sessionWindow.merge.sessions.in.local.partition to indicate Spark to perform partial aggregation.

#### Conditions for watermarking to clean aggregation state

It is important to note that the following conditions must be satisfied for the watermarking to clean the state in aggregation queries (as of Spark 2.1.1, subject to change in the future).

1. Output mode must be Append or Update. Complete mode requires all aggregate data to be preserved, and hence cannot use watermarking to drop intermediate state. See the Output Modes section for detailed explanation of the semantics of each output mode.

2. The aggregation must have either the event-time column, or a window on the event-time column.

3. withWatermark must be called on the same column as the timestamp column used in the aggregate. For example, df.withWatermark("time", "1 min").groupBy("time2").count() is invalid in Append output mode, as watermark is defined on a different column from the aggregation column.

4. withWatermark must be called before the aggregation for the watermark details to be used. For example, df.groupBy("time").count().withWatermark("time", "1 min") is invalid in Append output mode.

#### Semantic Guarantees of Aggregation with Watermarking

1. A watermark delay (set with withWatermark) of “2 hours” guarantees that the engine will never drop any data that is less than 2 hours delayed. In other words, any data less than 2 hours behind (in terms of event-time) the latest data processed till then is guaranteed to be aggregated.

2. However, the guarantee is strict only in one direction. Data delayed by more than 2 hours is not guaranteed to be dropped; it may or may not get aggregated. More delayed is the data, less likely is the engine going to process it.


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *

sparkSession = SparkSession.builder.config(conf=SparkConf() \
                            .setAppName('SS') \
                            .setMaster('local[4]')).enableHiveSupport().getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/10 12:26:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
schema = StructType([StructField('Transaction_ID', StringType(), True) \
            , StructField('Transaction_Date', StringType(), True) \
            , StructField('Transaction_Value', IntegerType(), True) \
            , StructField('Transaction_Segment', StringType(), True) \
            , StructField('Credit_Card_ID', StringType(), True) \
            , StructField('Age', IntegerType(), True) \
            , StructField('Gender', StringType(), True)])
data = sparkSession.readStream.schema(schema).csv('/Spark_streaming/TransactionBasenew2/')

In [ ]:
#thumbling window /fixed window
df = data.select('Transaction_ID',to_timestamp('Transaction_Date').alias('Transaction_Date'),'Transaction_Value','Transaction_Segment','Credit_Card_ID','Age','Gender')
df1 = df.groupBy(window('Transaction_Date', '10 minutes'), 'Gender').count()

query = df1 \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start(truncate=False)

query.awaitTermination()

23/01/10 11:55:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-637de597-a7b9-4097-ab0a-1a095750db26. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/10 11:55:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |33   |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|M     |122  |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|F     |42   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |223  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |205  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|F     |115  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+--------------------------------

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |67   |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|F     |77   |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |90   |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|F     |93   |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |113  |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|F     |117  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |135  |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|F     |132  |
+------------------------------------------+------+-----+



In [ ]:
#sliding window
df = data.select('Transaction_ID',to_timestamp('Transaction_Date').alias('Transaction_Date'),'Transaction_Value','Transaction_Segment','Credit_Card_ID','Age','Gender')
df1 = df.groupBy(window('Transaction_Date', '10 minutes','5 minutes'), 'Gender').count()

query = df1 \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start(truncate=False)

query.awaitTermination()

23/01/10 12:26:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f099e219-65fe-4bc9-8c71-b1bc6b65a52e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/10 12:26:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |280  |
|{2023-01-10 11:55:00, 2023-01-10 12:05:00}|M     |841  |
|{2023-01-10 12:15:00, 2023-01-10 12:25:00}|F     |50   |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |348  |
|{2023-01-09 17:45:00, 2023-01-09 17:55:00}|F     |320  |
|{2023-01-10 11:50:00, 2023-01-10 12:00:00}|M     |643  |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 16:55:00, 2023-01-09 17:05:00}|M     |1107 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 16:55:00, 2023-01-09 17:05:00}|F     |1062 |
|{2023-01-10 12:15:00, 2023-01-10 12:25:00}|M     |50   |
|{2023-01-10 12:00:00, 2023-01-10 12:10:00}|M     |198  |
|{2023-01-09 17:05:00, 2023-01-09

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |335  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |408  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |385  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |358  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |376  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |460  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |426  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |410  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |402  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |479  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |452  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |429  |
+------------------------------------------+------+-----+



...

-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |549  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |635  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |599  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |585  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 10
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |582  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |679  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |632  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |629  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 11
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |584  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |682  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |634  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |632  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 12
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |611  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |701  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |661  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |651  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 13
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|M     |622  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|F     |708  |
|{2023-01-10 12:20:00, 2023-01-10 12:30:00}|M     |672  |
|{2023-01-10 12:25:00, 2023-01-10 12:35:00}|F     |658  |
+------------------------------------------+------+-----+



In [4]:
df = data.groupBy('Gender').count()
query = df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

23/01/09 16:18:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bb9ebe81-cf8c-4c7c-a6f8-62223ef02be3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/09 16:18:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2595|
|     M| 2546|
+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2660|
|     M| 2610|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2689|
|     M| 2648|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2740|
|     M| 2693|
+------+-----+



...

-------------------------------------------
Batch: 9
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2934|
|     M| 2886|
+------+-----+



-------------------------------------------
Batch: 10
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2957|
|     M| 2911|
+------+-----+



-------------------------------------------
Batch: 11
-------------------------------------------
+------+-----+
|Gender|count|
+------+-----+
|     F| 2997|
|     M| 2958|
+------+-----+



ERROR:root:KeyboardInterrupt while sending command.              (90 + 4) / 200]
Traceback (most recent call last):
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/audacious/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df = data.select('Transaction_ID',to_timestamp('Transaction_Date').alias('Transaction_Date'),'Transaction_Value','Transaction_Segment','Credit_Card_ID','Age','Gender')
df1 = df.withWatermark('Transaction_Date', "30 minutes").groupBy(window('Transaction_Date', '10 minutes'), 'Gender').count()

query = df1 \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start(truncate=False)

query.awaitTermination()

23/01/09 17:48:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2a1f0771-2591-4b6e-9bbc-a12ec9ef18cb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/09 17:48:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |49   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |43   |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |80   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |74   |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |109  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |105  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



...

-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |177  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |160  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |223  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |205  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|M     |19   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |223  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |205  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|F     |22   |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|M     |26   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |223  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |205  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|F     |32   |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 10
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|M     |351  |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|F     |247  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|M     |83   |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|M     |223  |
|{2023-01-09 17:40:00, 2023-01-09 17:50:00}|F     |205  |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|F     |1399 |
|{2023-01-09 16:50:00, 2023-01-09 17:00:00}|F     |339  |
|{2023-01-09 17:50:00, 2023-01-09 18:00:00}|F     |80   |
|{2023-01-09 17:00:00, 2023-01-09 17:10:00}|M     |1453 |
|{2023-01-09 17:10:00, 2023-01-09 17:20:00}|M     |257  |
+------------------------------------------+------+-----+



In [ ]:
#session window
df = data.select('Transaction_ID',to_timestamp('Transaction_Date').alias('Transaction_Date'),'Transaction_Value','Transaction_Segment','Credit_Card_ID','Age','Gender')
df1 = df.withWatermark('Transaction_Date', "30 minutes").groupBy(session_window('Transaction_Date', '2 minutes'), 'Gender').count()

query = df1 \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start(truncate=False)

query.awaitTermination()

23/01/10 12:23:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-72617e8a-504d-467f-9339-6e5b20f9178a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/10 12:23:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:14}|F     |9    |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:14}|M     |9    |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:33}|F     |11   |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:33}|M     |14   |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:44}|F     |39   |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:44}|M     |39   |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:55}|F     |50   |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:26:55}|M     |50   |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:18}|F     |132  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:18}|M     |130  |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:29}|F     |164  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:29}|M     |163  |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+------+-----+
|session_window                            |Gender|count|
+------------------------------------------+------+-----+
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|F     |320  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:40}|F     |202  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|F     |1985 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|F     |833  |
|{2023-01-09 16:58:04, 2023-01-09 17:13:32}|M     |2061 |
|{2023-01-10 11:55:01, 2023-01-10 12:04:19}|M     |841  |
|{2023-01-10 12:24:14, 2023-01-10 12:27:40}|M     |210  |
|{2023-01-09 17:48:40, 2023-01-09 17:53:01}|M     |345  |
+------------------------------------------+------+-----+

